In [1]:
import requests
headers = {'User-Agent': 'Rishabh K Sharma/1.0 (Contact: sharmarishabhk@gmail.com)'}
response = requests.get('https://example.com', headers=headers)

response

<Response [200]>

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.tools.retriever import create_retriever_tool


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://www.myvi.in/blog/new-sim-card-rules")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

rules_tool = create_retriever_tool(retriever, 'New_connection_rules', 'Search for information about rules on buying a new sim card. for any question about new sim card rules use this tool')

In [4]:
loader = WebBaseLoader("https://www.myvi.in/blog/idea-fancy-number")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

fancy_number_tool = create_retriever_tool(retriever, 'fancy_number_guide', 'Search for information about guide on buying a new fancy number sim card. for any question about new fancy number sim card guide use this tool')

In [5]:
loader = WebBaseLoader("https://www.myvi.in/help-support/faqs/how-to-port-my-number-to-vi")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

port_number_tool = create_retriever_tool(retriever, 'port_number_guide', 'Search for information about guide on porting number to vi. for any question about mobile number portibility guide use this tool')

In [6]:
loader = WebBaseLoader("https://www.myvi.in/new-connection/buy-prepaid-sim-connection-online")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

FAQ_number_tool = create_retriever_tool(retriever, 'faq', 'Search for information about guide on frequently asked questions. for any question about new sim card connection use this tool')

In [7]:
tools = [rules_tool, fancy_number_tool, port_number_tool, FAQ_number_tool]

In [8]:
tools

[Tool(name='New_connection_rules', description='Search for information about rules on buying a new sim card. for any question about new sim card rules use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7b4c3c71f100>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b4c3c1e8fb0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7b4c3c7607c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b4c3c1e8fb0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types

In [9]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="Llama3.2:3B", temprature=0)

In [10]:
llm

ChatOllama(model='Llama3.2:3B')

In [11]:
from langchain_classic import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain_classic.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)


In [13]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [18]:
from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [19]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [20]:
agent_executor.invoke({"input": "Mujhe sim leni hai par mujhe pata nahi hai mai kya karu"})



> Entering new AgentExecutor chain...

Invoking: `faq` with `{'query': 'sim leni hai par mujhe pata nahi hai mai kya karu'}`


Get Vi New Prepaid SIM Card Online – Buy Mobile SIM Card In india 


























































































Back to CallCenter











Diwali deal: Get upto Rs 500 OFF on select prepaid plans
Diwali deal: Get upto Rs 500 OFF on select prepaid plans 


get a new prepaid SIM in 3 simple steps



select a pack




choose your number




Prepaid SIM confirmed


 





postpaid plans


family plans



prepaid packs





Please enter your pincode to view packs



please enter pincode










select a pack to start













no plan selected

show all plans







×













₹
325


excluding GST






              plan details
pack details 

 each secondary member gets



additional benefits



















I want this plan
I want this pack























₹



₹
329




validity



calls







{'input': 'Mujhe sim leni hai par mujhe pata nahi hai mai kya karu',
 'output': 'मुझे खेद है, लेकिन मैंने आपको यह स्पष्ट करना चाहूंगा कि आप एक विशिष्ट प्रश्न पूछ रहे थे, जिसका उत्तर देने में मैं सक्षम नहीं हुआ। लेकिन मैं आपकी सहायता के लिए यहाँ हूँ।\n\nआपके पूछे गए प्रश्न का सारांश यह है: "मुझे सिम लेनी है, लेकिन मुझे पता नहीं है कि मैं क्या करूँ।"\n\nयहाँ कुछ सुझाव दिए गए हैं जो आपको अपने सिम लेने में मदद कर सकते हैं:\n\n1. **वाईडफोन आईडिया वेबसाइट पर जाएं**: आप वाईडफोन आईडिया की आधिकारिक वेबसाइट पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको विभिन्न प्लान और पैक के विकल्प मिलेंगे।\n2. **स्टोर में जाएं**: आप वाईडफोन आईडिया के निकटतम स्टोर पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको सहायता प्रदान करने वाले कर्मचारी मिलेंगे।\n3. **ऑटो-रिचर्ज टेलीफोन पर कॉल करें**: आप वाईडफोन आईडिया के ऑटो-रिचर्ज टेलीफोन पर कॉल करके अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं।\n4. **सोशल मीडिया पर जानकारी प्राप्त करें**: आप वाईडफोन आईडिया के आधिकारिक सोशल मीडिया अकाउंट्

In [38]:
data = {'input': 'Mujhe sim leni hai par mujhe pata nahi hai mai kya karu',
 'output': 'मुझे खेद है, लेकिन मैंने आपको यह स्पष्ट करना चाहूंगा कि आप एक विशिष्ट प्रश्न पूछ रहे थे, जिसका उत्तर देने में मैं सक्षम नहीं हुआ। लेकिन मैं आपकी सहायता के लिए यहाँ हूँ।\n\nआपके पूछे गए प्रश्न का सारांश यह है: "मुझे सिम लेनी है, लेकिन मुझे पता नहीं है कि मैं क्या करूँ।"\n\nयहाँ कुछ सुझाव दिए गए हैं जो आपको अपने सिम लेने में मदद कर सकते हैं:\n\n1. **वाईडफोन आईडिया वेबसाइट पर जाएं**: आप वाईडफोन आईडिया की आधिकारिक वेबसाइट पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको विभिन्न प्लान और पैक के विकल्प मिलेंगे।\n2. **स्टोर में जाएं**: आप वाईडफोन आईडिया के निकटतम स्टोर पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको सहायता प्रदान करने वाले कर्मचारी मिलेंगे।\n3. **ऑटो-रिचर्ज टेलीफोन पर कॉल करें**: आप वाईडफोन आईडिया के ऑटो-रिचर्ज टेलीफोन पर कॉल करके अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं।\n4. **सोशल मीडिया पर जानकारी प्राप्त करें**: आप वाईडफोन आईडिया के आधिकारिक सोशल मीडिया अकाउंट्स पर जाकर अपने सिम लेने की प्रक्रिया के बारे में जानकारी प्राप्त कर सकते हैं।\n\nमुझे आशा है कि ये सुझाव आपको अपने सिम लेने में मदद करेंगे। यदि आपको और अधिक सहायता की आवश्यकता है, तो कृपया मुझसे पूछें।'}
display(Markdown(f""" ### User - {data['input']} \n\n ### Agent - \n{data['output']}"""))

 ### User - Mujhe sim leni hai par mujhe pata nahi hai mai kya karu 

 ### Agent - 
मुझे खेद है, लेकिन मैंने आपको यह स्पष्ट करना चाहूंगा कि आप एक विशिष्ट प्रश्न पूछ रहे थे, जिसका उत्तर देने में मैं सक्षम नहीं हुआ। लेकिन मैं आपकी सहायता के लिए यहाँ हूँ।

आपके पूछे गए प्रश्न का सारांश यह है: "मुझे सिम लेनी है, लेकिन मुझे पता नहीं है कि मैं क्या करूँ।"

यहाँ कुछ सुझाव दिए गए हैं जो आपको अपने सिम लेने में मदद कर सकते हैं:

1. **वाईडफोन आईडिया वेबसाइट पर जाएं**: आप वाईडफोन आईडिया की आधिकारिक वेबसाइट पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको विभिन्न प्लान और पैक के विकल्प मिलेंगे।
2. **स्टोर में जाएं**: आप वाईडफोन आईडिया के निकटतम स्टोर पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको सहायता प्रदान करने वाले कर्मचारी मिलेंगे।
3. **ऑटो-रिचर्ज टेलीफोन पर कॉल करें**: आप वाईडफोन आईडिया के ऑटो-रिचर्ज टेलीफोन पर कॉल करके अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं।
4. **सोशल मीडिया पर जानकारी प्राप्त करें**: आप वाईडफोन आईडिया के आधिकारिक सोशल मीडिया अकाउंट्स पर जाकर अपने सिम लेने की प्रक्रिया के बारे में जानकारी प्राप्त कर सकते हैं।

मुझे आशा है कि ये सुझाव आपको अपने सिम लेने में मदद करेंगे। यदि आपको और अधिक सहायता की आवश्यकता है, तो कृपया मुझसे पूछें।

In [21]:
agent_executor.invoke({"input": "I want to buy a Vi Postpaid sim, what is the process? how should i proceed in order to get this super fast?"})



> Entering new AgentExecutor chain...

Invoking: `port_number_guide` with `{'query': 'Vi Postpaid sim porting process and procedure for speedy connection'}`


Logout
























New to Vi






















I want to port my number to Vi
 
Great choice! Porting to Vi is pain-free and convenient using Mobile Number Portability (MNP)
 

Please generate a UPC code by sending PORT to 1900 from your existing network provider. Kindly clear all your dues before porting. 
You can then order a new Vi connection online and get SIM delivery to your doorstep by simply sharing your details here. Port SIM!
Select from postpaid plans/prepaid plans, and get OTP to complete your order.

 
Also, you can visit your nearest Vi store to buy a new connection.
Please carry your valid Identity Proof and Address Proof with you.
To find your nearest store, Click here.








Go Back









Related Searches

I want to convert my Prepaid to Postpaid connection
I want to buy fancy mobile number

{'input': 'I want to buy a Vi Postpaid sim, what is the process? how should i proceed in order to get this super fast?',
 'output': 'To port your number to Vi, you can follow these steps:\n\n1. **Generate a UPC code**: Send the message "PORT" to 1900 from your existing network provider to generate a Unique Porting Code (UPC).\n2. **Clear all dues**: Ensure that you clear all your outstanding balances before porting your number.\n3. **Order a new Vi connection online**: Go to the Vi website and order a new postpaid or prepaid connection. You will receive an OTP to complete the process.\n4. **Share your details**: Share your personal details, such as name, address, and mobile number, with Vi to facilitate SIM delivery.\n5. **Get SIM delivery**: Vi will deliver your SIM card to your doorstep.\n6. **Activate your new connection**: Activate your new connection by following the in-app instructions or by visiting a nearby Vi store.\n\nBefore proceeding, please ensure that you have the necessa

In [28]:
from IPython.display import display, Markdown

def get_markdown(data):
    markdown_text = f"""
        ### User Input
        
        {data['input']}
        
        ### System Output
        
        {data['output']}
    """
    return Markdown(markdown_text)


In [37]:
data = {'input': 'I want to buy a Vi Postpaid sim, what is the process? how should i proceed in order to get this super fast?',
 'output': 'To port your number to Vi, you can follow these steps:\n\n1. **Generate a UPC code**: Send the message "PORT" to 1900 from your existing network provider to generate a Unique Porting Code (UPC).\n2. **Clear all dues**: Ensure that you clear all your outstanding balances before porting your number.\n3. **Order a new Vi connection online**: Go to the Vi website and order a new postpaid or prepaid connection. You will receive an OTP to complete the process.\n4. **Share your details**: Share your personal details, such as name, address, and mobile number, with Vi to facilitate SIM delivery.\n5. **Get SIM delivery**: Vi will deliver your SIM card to your doorstep.\n6. **Activate your new connection**: Activate your new connection by following the in-app instructions or by visiting a nearby Vi store.\n\nBefore proceeding, please ensure that you have the necessary documents ready, such as:\n\n* Valid identity proof (e.g., Aadhaar card, pan card)\n* Address proof (e.g., electricity bill, bank statement)\n\nAdditionally, you can visit your nearest Vi store to purchase a new connection and get assistance with the porting process.'}
display(Markdown(f""" ### User - {data['input']} \n\n ### Agent - \n{data['output']}"""))

 ### User - I want to buy a Vi Postpaid sim, what is the process? how should i proceed in order to get this super fast? 

 ### Agent - 
To port your number to Vi, you can follow these steps:

1. **Generate a UPC code**: Send the message "PORT" to 1900 from your existing network provider to generate a Unique Porting Code (UPC).
2. **Clear all dues**: Ensure that you clear all your outstanding balances before porting your number.
3. **Order a new Vi connection online**: Go to the Vi website and order a new postpaid or prepaid connection. You will receive an OTP to complete the process.
4. **Share your details**: Share your personal details, such as name, address, and mobile number, with Vi to facilitate SIM delivery.
5. **Get SIM delivery**: Vi will deliver your SIM card to your doorstep.
6. **Activate your new connection**: Activate your new connection by following the in-app instructions or by visiting a nearby Vi store.

Before proceeding, please ensure that you have the necessary documents ready, such as:

* Valid identity proof (e.g., Aadhaar card, pan card)
* Address proof (e.g., electricity bill, bank statement)

Additionally, you can visit your nearest Vi store to purchase a new connection and get assistance with the porting process.

In [23]:
agent_executor.invoke({"input": "Can i get somebody at my home or office to do all this stuff"})



> Entering new AgentExecutor chain...

Invoking: `faq` with `{'query': 'can i get somebody at my home or office to do all this stuff'}`


Prepaid Connection
                            





                                Track Your SIM
                            





                                SIM Card Home Delivery
                            





                                Best Postpaid Plans
                            





                                New SIM Card
                            





                                Family Plans
                            





                                Vanity Number
                            





This website uses own third-party cookies. Find out more about usage in our Privacy Policy page. Copyright Reserved with Vodafone Idea Limited (formerly Idea Cellular Limited).
Vodafone Idea Limited (Formerly Idea Cellular Limited), An Aditya Birla Group & Vodafone partnership, Suman Towers, Plot No.18, Sector 1

{'input': 'Can i get somebody at my home or office to do all this stuff',
 'output': "To get someone at your home or office to do all the tasks related to buying a new prepaid connection, you have a few options:\n\n1. **Visit a Vodafone Idea Store**: You can visit a nearby Vodafone Idea store along with the required documents (Proof of Address and Proof of Identity) and get a new prepaid SIM card activated.\n2. **Order Online**: You can order a new prepaid SIM online through the Vodafone Idea website or mobile app, and then collect it from a nearby store or have it delivered to your home address.\n3. **Use the 'My Account' Portal**: If you already have a Vodafone Idea connection, you can log in to your account portal, select 'SIMs', and follow the instructions to order a new SIM card online.\n4. **Call Customer Care**: You can also call the Vodafone Idea customer care number (1860-266-6767) and speak to a representative to place an order for a new prepaid SIM card.\n\nTo get someone at

In [36]:
data = {'input': 'Can i get somebody at my home or office to do all this stuff',
 'output': "To get someone at your home or office to do all the tasks related to buying a new prepaid connection, you have a few options:\n\n1. **Visit a Vodafone Idea Store**: You can visit a nearby Vodafone Idea store along with the required documents (Proof of Address and Proof of Identity) and get a new prepaid SIM card activated.\n2. **Order Online**: You can order a new prepaid SIM online through the Vodafone Idea website or mobile app, and then collect it from a nearby store or have it delivered to your home address.\n3. **Use the 'My Account' Portal**: If you already have a Vodafone Idea connection, you can log in to your account portal, select 'SIMs', and follow the instructions to order a new SIM card online.\n4. **Call Customer Care**: You can also call the Vodafone Idea customer care number (1860-266-6767) and speak to a representative to place an order for a new prepaid SIM card.\n\nTo get someone at your home or office to do all these tasks, you can consider hiring a service that offers prepaid connection setup services. These services typically include:\n\n* Visiting the Vodafone Idea store on your behalf\n* Placing an online order and selecting the desired plan\n* Ordering additional services (e.g., data packs, international calls)\n* Activating the SIM card\n\nYou can search for such services in your area by searching online or checking local directories. Some popular options include:\n\n* **Home setup services**: Companies like Home Setup Services or ConnectMe offer home setup services where they send a technician to set up your new prepaid connection.\n* **Prepaid SIM activation services**: Companies like Sim Activation Service or Prepaid SIM Activate offer prepaid SIM activation services, including ordering and activating a new SIM card online.\n\nKeep in mind that these services may charge a fee for their services, so be sure to compare prices and services before hiring someone."}
display(Markdown(f""" ### User - {data['input']} \n\n ### Agent - \n{data['output']}"""))

 ### User - Can i get somebody at my home or office to do all this stuff 

 ### Agent - 
To get someone at your home or office to do all the tasks related to buying a new prepaid connection, you have a few options:

1. **Visit a Vodafone Idea Store**: You can visit a nearby Vodafone Idea store along with the required documents (Proof of Address and Proof of Identity) and get a new prepaid SIM card activated.
2. **Order Online**: You can order a new prepaid SIM online through the Vodafone Idea website or mobile app, and then collect it from a nearby store or have it delivered to your home address.
3. **Use the 'My Account' Portal**: If you already have a Vodafone Idea connection, you can log in to your account portal, select 'SIMs', and follow the instructions to order a new SIM card online.
4. **Call Customer Care**: You can also call the Vodafone Idea customer care number (1860-266-6767) and speak to a representative to place an order for a new prepaid SIM card.

To get someone at your home or office to do all these tasks, you can consider hiring a service that offers prepaid connection setup services. These services typically include:

* Visiting the Vodafone Idea store on your behalf
* Placing an online order and selecting the desired plan
* Ordering additional services (e.g., data packs, international calls)
* Activating the SIM card

You can search for such services in your area by searching online or checking local directories. Some popular options include:

* **Home setup services**: Companies like Home Setup Services or ConnectMe offer home setup services where they send a technician to set up your new prepaid connection.
* **Prepaid SIM activation services**: Companies like Sim Activation Service or Prepaid SIM Activate offer prepaid SIM activation services, including ordering and activating a new SIM card online.

Keep in mind that these services may charge a fee for their services, so be sure to compare prices and services before hiring someone.